## Pemetaan Hubungan Perilaku antar Pengguna

### Library yang Dibutuhkan

Melakukan import library yang dibutuhkan

In [ ]:
pip install association-metrics

In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import scipy.stats as scp
import association_metrics as am

from sklearn.preprocessing import LabelEncoder 
from collections import OrderedDict
from scipy.stats import chisquare
from sklearn.feature_selection import chi2

Melakukan koneksi sumber data ke Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load Data Periode Sebelum dan Setelah COVID-19

Melakukan load data pengguna aktif pada periode sebelum dan setelah COVID-19

In [ ]:
#before covid
data_df = pd.read_csv('/content/drive/MyDrive/Thesis/output/4_07022022_noncovid_ekstraksi_variabel_perilaku.csv')

#after covid
#data_df = pd.read_csv('/content/drive/MyDrive/Thesis/output/4_07022022_covid_ekstraksi_variabel_perilaku.csv')

Menghitung jumlah data penggun aktif

In [ ]:
len(data_df)

23647

### Konversi dan Perhitungan Populasitas Pengguna berdasarkan Tweet

Melakukan konvert kolom label emosi ke dalam bentuk Integer

In [ ]:
#labelencoder= LabelEncoder() #initializing an object of class LabelEncoder
#data_df['emotion'] = labelencoder.fit_transform(data_df['emotion']) #fitting and transforming the desired categorical column.

#data_df = data_df.replace(to_replace ="anger", value = 1)
#data_df = data_df.replace(to_replace ="happy", value = 2)
#data_df = data_df.replace(to_replace ="sadness", value = 3)
#data_df = data_df.replace(to_replace ="fear", value = 4)
#data_df = data_df.replace(to_replace ="love", value = 5)

Menentukan tweet popularitas berdasarkan jumlah like, jumlah retweet dan jumlah reply

In [ ]:
data_df['replies_count'].replace(to_replace = '-6.216657128974757,106.83030289065285,10km', value = 0, inplace = True)

data_df['likes_count'] = data_df['likes_count'].fillna(0).astype(int)
data_df['replies_count'] = data_df['replies_count'].fillna(0).astype(int)
data_df['retweets_count'] = data_df['retweets_count'].fillna(0).astype(int)
#data_df['emotion'] = data_df['emotion'].fillna(0).astype(int)

tweet_popularity_label = []

for i, data in data_df.iterrows():
  tweet_popularity = (data['replies_count'] + data['likes_count'] + data['retweets_count'])

  tweet_popularity_label.append(tweet_popularity)

data_df['tweet_popularity'] = tweet_popularity_label

Menampilkan jumlah pengguna aktif berdasarkan hasi perhitungan popularitas tweet

In [ ]:
users = data_df['username'].unique().tolist()

In [ ]:
#data_df['cluster'][data_df.tweet_id == 1227612537065074688] = 2
#data_df['cluster'][data_df.tweet_id == 1227439087528882178] = 2
#data_df['cluster'][data_df.tweet_id == 1226368440966897664] = 2
#data_df['cluster'][data_df.tweet_id == 1226368286054502402] = 2

data_df = data_df.loc[:, ~data_df.columns.str.contains('^Unnamed')]

### Melakukan Pemetaan Similaritas Perilaku

Melakukan deklarasi persamaan Cramer's V

In [ ]:
#def cramers_V(var1, var2) :
#  crosstab =np.array(pd.crosstab(var1, var2, rownames=None, colnames=None)) # Cross table building
#  stat = scp.chi2_contingency(crosstab)[0] # Keeping of the test statistic of the Chi2 test
#  obs = np.sum(crosstab) # Number of observations
#  mini = min(crosstab.shape)-1 # Take the minimum value between the columns and the rows of the cross table
  
 # return (stat/(obs*mini))

Melakukan proses pemetaan perilaku dari kombinasi pengguna aktif berdasarkan Persamaan Cramer's V

In [ ]:
#from scipy.stats import pearsonr

relation_data = []

for vi in users:
    for vj in users:
        if vi != vj:
            data_vi = data_df[data_df['username'] == vi]
            data_vj = data_df[data_df['username'] == vj]
            
            len_data = min(len(data_vi),len(data_vj))

            if len_data > 100:
            #if len_data > 50:
              #seleksi data berdasarkan tingginya popularitas jumlah like, jumlah retweet dan jumlah reply
              final_data_vi = data_vi.sort_values(by=['tweet_popularity'], ascending=False).head(len_data)
              final_data_vj = data_vj.sort_values(by=['tweet_popularity'], ascending=False).head(len_data)

              behavior_df = pd.DataFrame()
              behavior_df['cluster_user1'] = final_data_vi['cluster'].tolist()
              behavior_df['emotion_user1'] = final_data_vi['emotion'].tolist()

              behavior_df['cluster_user2'] = final_data_vj['cluster'].tolist()
              behavior_df['emotion_user2'] = final_data_vj['emotion'].tolist()

              #convert column to str
              behavior_df['cluster_user1'] = behavior_df['cluster_user1'].values.astype(str)
              behavior_df['cluster_user2'] = behavior_df['cluster_user2'].values.astype(str)
              #behavior_df['topic_predict_user1'] = behavior_df['topic_predict_user1'].values.astype(str)
              #behavior_df['topic_predict_user2'] = behavior_df['topic_predict_user2'].values.astype(str)

              # Convert you str columns to Category columns
              behavior_df = behavior_df.apply(
                                lambda x: x.astype("category") if x.dtype == "O" else x)
              
              #topic_corr = cramers_V(behavior_df['cluster_user1'], behavior_df['cluster_user2'])
              #emotion_corr = cramers_V(behavior_df['emotion_user1'], behavior_df['emotion_user2'])

              # Initialize a CamresV object using you pandas.DataFrame
              
              #topic
              try:
                topic_cramersv = am.CramersV(behavior_df[['cluster_user1', 'cluster_user2']])
                #topic_corr = cramers_V(behavior_df[['cluster_user1', 'cluster_user2']])
                # will return a pairwise matrix filled with Cramer's V, where columns and index are 
                # the categorical variables of the passed pandas.DataFrame
                topic_corr = topic_cramersv.fit()
                topic_corr = topic_corr.iloc[0, 1]
              except:
                topic_corr = 0
                #print("{}-{}".format(vi, vj))

              #emotion
              try:
                emotion_cramersv = am.CramersV(behavior_df[['emotion_user1', 'emotion_user2']])
                #emotion_corr = cramers_V(behavior_df[['emotion_user1', 'emotion_user2']])
                # will return a pairwise matrix filled with Cramer's V, where columns and index are 
                # the categorical variables of the passed pandas.DataFrame
                emotion_corr = emotion_cramersv.fit()
                emotion_corr = emotion_corr.iloc[0, 1]
              except:
                emotion_corr = 0
                #print("emotion")
              
              if topic_corr > 0.25 or emotion_corr > 0.25:
              #if topic_corr > 0.25 and emotion_corr > 0.25:
                  similarity_variable = max(topic_corr, emotion_corr)
                  if(similarity_variable == topic_corr):
                      relation_data.append([vi, len(data_vi), vj, len(data_vj), topic_corr, "topic"])
                  elif (similarity_variable == emotion_corr):
                      relation_data.append([vi, len(data_vi), vj, len(data_vj), emotion_corr, "emotion"])
                  #relation_data.append([vi, len(data_vi), vj, len(data_vj), emotion_corr, topic_corr])

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/pivot.py:188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agged[v] = maybe_downcast_to_dtype(agged[v], data[v].dtype)


In [ ]:
len(relation_data)

634

### Konversi Hasil Pemetaan Similaritas Perilaku Pengguna

Melakukan konversi hasil pemetaan similaritas perilaku ke dalam sebuah tabel DataFrame

In [ ]:
result_data = pd.DataFrame(relation_data, columns=['from_user', 'from_user_tweet_number', 'to_user', 'to_user_tweet_number', 'relation_value', 'relation_type'])

In [ ]:
labelencoder= LabelEncoder() #initializing an object of class LabelEncoder
result_data['from_user_alias'] = result_data['from_user']
result_data['from_user_alias'] = labelencoder.fit_transform(result_data['from_user_alias']) 
#fitting and transforming the desired categorical column.

Menampilkan hasil similaritas perilaku pengguna

In [ ]:
result_data

,from_user,from_user_tweet_number,to_user,to_user_tweet_number,relation_value,relation_type,from_user_alias
0,3le4n012,235,cyfranz,101,0.274448,topic,0
1,3le4n012,235,isar____,146,0.254276,topic,0
2,3le4n012,235,jodi_wirawan,170,0.290601,topic,0
3,3le4n012,235,lisasarilisa,103,0.361930,topic,0
4,3le4n012,235,migumogu_,168,0.273407,topic,0
...,...,...,...,...,...,...,...
629,yuliani2907,130,cagubnyinyir2,300,0.259802,emotion,97
630,yuliani2907,130,kwonfire3424,155,0.278908,emotion,97
631,yuliani2907,130,lovelyndelight,296,0.275543,topic,97
632,yuliani2907,130,m_jahroni,274,0.267136,topic,97


Melakukan penyimpanan data hasil similaritas perilaku pengguna

In [ ]:
to_user_label = []

for i, data in result_data.iterrows():
  resulted_data = result_data[result_data['from_user'] == data['to_user']]['from_user_alias'].drop_duplicates()

  to_user_label.append(resulted_data.iloc[0])

result_data['to_user_alias'] = to_user_label

In [ ]:
result_data.to_csv('/content/drive/MyDrive/Thesis/output/6_04042022_before_covid_pearson_result.csv')
#result_data.to_csv('/content/drive/MyDrive/Thesis/output/6_04042022_after_covid_pearson_result.csv')